# <center>Quickstart Guide</center>
## <center>Gain insights into your model via Phoenix</center>

Phoenix first and foremost is an application that can run alongside your notebook environment. It takes in up to two sets of data and surfaces up drift, performance, and data quality insights.


### 📚 Install `arize-phoenix` 

In [ ]:
import phoenix as px

In [ ]:
?px

### Using a built-in dataset to view the application

To get familiar with the application itself, the easiest way to get started is to use one of phoenix's example datasets.

In [ ]:
import phoenix as px

# Get the fixture datasets via a specific use case. Some valid values are "fashion_mnist", "sentiment_classification_language_drift", and "credit_card_fraud"
datasets = px.load_example("sentiment_classification_language_drift")
session = px.launch_app(datasets.primary, datasets.reference)
session.view()

INFO:     127.0.0.1:58714 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:58714 - "GET /index.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:58716 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     127.0.0.1:58714 - "POST /graphql HTTP/1.1" 200 OK
INFO:     127.0.0.1:58714 - "POST /graphql HTTP/1.1" 200 OK
node: EmbeddingDimension 0
INFO:     127.0.0.1:58714 - "POST /graphql HTTP/1.1" 200 OK
node: EmbeddingDimension 0
INFO:     127.0.0.1:58714 - "POST /graphql HTTP/1.1" 200 OK
node: EmbeddingDimension 0
INFO:     127.0.0.1:58716 - "POST /graphql HTTP/1.1" 200 OK
